<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_TensorFlow/blob/main/01LinearNeuralNetWorkForRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class SGD():
    """Minibatch stochastic gradient descent."""
    def __init__(self, lr):
      self.lr = lr

    def apply_gradients(self, grads_and_vars):
        for grad, param in grads_and_vars:
            param.assign_sub(self.lr * grad)

In [3]:
class LinearRegressionScratch(tf.keras.Model):
    """The linear regression model implemented from scratch."""
    def __init__(self, num_inputs, lr, sigma=0.01):
        super().__init__()
        self.lr = lr
        w = tf.random.normal((num_inputs, 1), mean=0, stddev=0.01)
        b = tf.zeros(1)
        self.w = tf.Variable(w, trainable=True)
        self.b = tf.Variable(b, trainable=True)

    def forward(self, X):
        return tf.matmul(X, self.w) + self.b

    def custom_mse_loss(self, y_hat, y):
        l = (y_hat - y) ** 2 / 2
        return tf.reduce_mean(l)

    def configure_optimizers(self):
        return SGD(self.lr)

    def prepare_batch(self, batch):
        return batch

    def training_step(self, batch):
        X_batch, y_batch = batch
        y_hat = self.forward(X_batch)
        loss = self.custom_mse_loss(y_hat, y_batch) # Calling the renamed loss method
        return loss

    # New fit method to encapsulate the training loop
    def fit(self, dataset, num_epochs):
        optimizer = self.configure_optimizers()
        for epoch in range(num_epochs):
            last_batch_loss = 0.0 # To store the loss of the last batch in the epoch
            for batch in dataset:
                with tf.GradientTape() as tape:
                    loss = self.training_step(batch)

                grads = tape.gradient(loss, [self.w, self.b])
                optimizer.apply_gradients(zip(grads, [self.w, self.b]))
                last_batch_loss = loss.numpy()
            print(f"Epoch {epoch+1}, loss = {last_batch_loss:.4f}")

    # Removed the old fit_epoch method as it was designed for a Trainer class
    # def fit_epoch(self):
    #     self.model.training = True
    #     for batch in self.train_dataloader:
    #         with tf.GradientTape() as tape:
    #             loss = self.model.training_step(self.prepare_batch(batch))
    #         grads = tape.gradient(loss, self.model.trainable_variables)
    #         if self.gradient_clip_val > 0:
    #             grads = self.clip_gradients(self.gradient_clip_val, grads)
    #         self.optim.apply_gradients(zip(grads, self.model.trainable_variables))
    #         self.train_batch_idx += 1
    #     if self.val_dataloader is None:
    #         return
    #     self.model.training = False
    #     for batch in self.val_dataloader:
    #         self.model.validation_step(self.prepare_batch(batch))
    #         self.val_batch_idx += 1

In [4]:
import tensorflow as tf

# True parameters
true_w = tf.constant([[2.0], [-3.4]])
true_b = 4.2

# Generate synthetic data
num_samples = 2000
X = tf.random.normal((num_samples, 2))
y = tf.matmul(X, true_w) + true_b
y += tf.random.normal(y.shape, stddev=0.01)  # noise

dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(1000).batch(32)

In [5]:
model = LinearRegressionScratch(num_inputs=2, lr=0.03)

# Define the number of epochs (as used in the previous training run)
num_epochs = 10

# Call the new fit method on the model
model.fit(dataset, num_epochs)

Epoch 1, loss = 0.3561
Epoch 2, loss = 0.0125
Epoch 3, loss = 0.0002
Epoch 4, loss = 0.0001
Epoch 5, loss = 0.0000
Epoch 6, loss = 0.0000
Epoch 7, loss = 0.0001
Epoch 8, loss = 0.0001
Epoch 9, loss = 0.0001
Epoch 10, loss = 0.0000


With add_weights

In [6]:
class LinearRegression(tf.keras.Model):
  def __init__(self, number_inputs):
      super().__init__()
      self.w = self.add_weight(shape = (number_inputs, 1), initializer = 'glorot_uniform', trainable = True)
      self.b = self.add_weight(shape = (1,), initializer = 'zeros', trainable = True)

  def call(self, X):
    return tf.matmul(X, self.w) + self.b

In [7]:
model = LinearRegression(2)

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss = 'mse', metrics=['mae'])

In [9]:
# Train/test split
X_train, X_test = X[:1500], X[1500:]
y_train, y_test = y[:1500], y[1500:]

In [10]:
model.fit(X_train, y_train, epochs = 100)

Epoch 1/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 29.4913 - mae: 4.5722
Epoch 2/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 27.1511 - mae: 4.4168
Epoch 3/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.5789 - mae: 4.3893
Epoch 4/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.3305 - mae: 4.3346
Epoch 5/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25.6516 - mae: 4.2456
Epoch 6/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25.1580 - mae: 4.2372
Epoch 7/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.5774 - mae: 4.1273
Epoch 8/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.4734 - mae: 4.1285
Epoch 9/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.1838 - mae: 4.0808
Epoch 10/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.0650 - mae: 3.9507
Epoch 11/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.6052 - mae: 4.0094
Epoch 12/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21.1266 - mae: 3.8754
Epoch 13/100
47/47 ━━━━━━

In [11]:
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

In [12]:
print(f'Test loss {test_loss:.4f}, Test Mae {test_mae:.4f}')

Test loss 0.4976, Test Mae 0.6578


In [13]:
x_new = tf.constant([[1.5, 0.0]], dtype=tf.float32) # Added a second feature
print("Prediction for x=1.5 and x_2=0.0:", model(x_new).numpy())

Prediction for x=1.5 and x_2=0.0: [[6.5578375]]


With Dense

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
class LinearRegressionDense(tf.keras.Model):
  def __init__(self, number_inputs):
      super().__init__()
      self.dense_layer = Dense(units=1, input_shape=(number_inputs,), activation=None,
                               kernel_initializer='glorot_uniform', # Corresponds to glorot_uniform
                               bias_initializer='zeros') # Corresponds to zeros

  def call(self, X):
    return self.dense_layer(X)

# Example usage:
model_dense = LinearRegressionDense(2)
model_dense.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
model_dense.fit(X_train, y_train, epochs=100)

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 38.8303 - mae: 5.1213
Epoch 2/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40.1816 - mae: 5.2069
Epoch 3/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.1140 - mae: 4.9987
Epoch 4/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.8520 - mae: 5.0522
Epoch 5/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35.8858 - mae: 4.9260
Epoch 6/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35.8483 - mae: 4.8862
Epoch 7/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.8647 - mae: 4.8548
Epoch 8/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.4258 - mae: 4.6448
Epoch 9/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.4739 - mae: 4.6841
Epoch 10/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.4255 - mae: 4.5943
Epoch 11/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.1084 - mae: 4.5912
Epoch 12/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.5848 - mae: 4.5251
Epoch 13/100
47/47 ━━━━━━━━━━━━━━━━━━

Sequential

In [16]:
model = Sequential([Dense(1, input_shape=(2,))])

In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

In [18]:
model.fit(X_train, y_train, epochs = 100)

Epoch 1/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 37.7536 - mae: 4.9857
Epoch 2/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38.2957 - mae: 5.0531
Epoch 3/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.4170 - mae: 4.9479
Epoch 4/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.3586 - mae: 4.9668
Epoch 5/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.7349 - mae: 4.9593
Epoch 6/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.1782 - mae: 4.6063
Epoch 7/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.6452 - mae: 4.7034
Epoch 8/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.0161 - mae: 4.6695
Epoch 9/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.4233 - mae: 4.5482
Epoch 10/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.1290 - mae: 4.5305
Epoch 11/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.9092 - mae: 4.4971
Epoch 12/100
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 30.2699 - mae: 4.5254
Epoch 13/100
47/47 ━━━━━━